# PCA
PCA steps for data downscaling projet, dominant PCs to be exported in an array since that is all we really work with in the main downscalling file (in an effort to reduce memory load)<br>

World data preprocessing in this file because i didnt feel as if a new netCDF file was necessary for that data as it already loads rather quickly and doesn't take up much space

In [ ]:
#load in HRDPS and world data (for now still just working on pressure)
P_world= xr.open_dataset("psl_NAM-22_CCCma-CanESM2_rcp45_r1i1p1_CCCma-CanRCM4_r2_day_20160101-20201231.nc")
hrdps = xr.open_dataset('HRDPSsubset.nc')

In [ ]:
#first lets do some quick processing of the hrdps data
hrdps_lat = hrdps.nav_lat.values
hrdps_lon = hrdps.nav_lon.values
p_temp = hrdps.slp.values

In [ ]:
#get them into a shape we can work in
hrdps_lat = hrdps_lat[0,:,:].flatten()
hrdps_lon = hrdps_lon[0,:,:].flatten()

P_hrdps = np.empty((np.shape(p_temp)[1]*np.shape(p_temp)[2],np.shape(p_temp)[0]))
for i in range(np.shape(p_temp)[0]):
    P_hrdps[:,i] = np.reshape(p_temp[i],(np.shape(p_temp)[1]*np.shape(p_temp)[2],))

In [ ]:
#replace all nan with 101325
P_hrdps = (pd.DataFrame(P_hrdps).fillna(101325)).to_numpy() 

In [ ]:
#now for preprocessing of world data

#trim the world dataset to be for right time extent
#want days between June 1 2018 and June 30 2018
P_world = P_world.sel(time=slice('2016-01-01 12:00:00', '2019-12-31 12:00:00'))

In [ ]:
#now extract the data you want from the CanRCM4 xarray and trim them according to lat and lon of the HRDPS data
#first decide on range you want to work within for CanRCM4 data, want to overlap the HRDPS data by 10% of the max distance (y)
buffer = (max(hrdps_lat)-min(hrdps_lat))*0.1

lon = P_world.lon.values.flatten()
lat = P_world.lat.values.flatten()
index = []

#first find idexes that fit into lon range and lat range
for i in range(len(lon)):
    if lon[i] > (min(hrdps_lon)-buffer) and lon[i] < (max(hrdps_lon)+buffer) and lat[i] > (min(hrdps_lat)-buffer) and lat[i] < (max(hrdps_lat)+buffer):
        index.append(i)

#now make new lat, and lon
lat_RCM = []
lon_RCM = []

for i in index:
    lat_RCM.append(lat[i])
    lon_RCM.append(lon[i])

In [ ]:
#now convert data to 2D and use the idices found in the previous step to trim it
P2d = np.empty((np.shape(P_world.psl.values)[1]*np.shape(P_world.psl.values)[2],np.shape(P_world.psl.values)[0]))
for i in range(np.shape(P_world.psl.values)[0]):
    P2d[:,i] = np.reshape(P_world.psl.values[i],(np.shape(P_world.psl.values)[1]*np.shape(P_world.psl.values)[2],))

P_RCM = np.empty((len(index),np.shape(P_world.psl.values)[0]))

for i in range(np.shape(P_hrdps)[1]):
    for j in range(len(index)):
        P_RCM[j,i] = P2d[index[j],i]

## PCA
NOTE : before export find out how to limit range of the PCs so that you don't need to normalize afterwords

In [ ]:
#PCA on the world data
#looking for dominant spatial patterns to be eigenvectors and how those spatial patterns evelove over the month to the the PCs)
#need to take the transpose of the matrix 

data = P_RCM.T

n_modes = np.min(np.shape(data))
pca = PCA(n_components = n_modes)
RCM_PCs = pca.fit_transform(data)
RCM_eigvecs = pca.components_
RMC_fracVar = pca.explained_variance_ratio_

In [ ]:
#export the relevant arrays to CSV
numpy.savetxt("CanRCM4_PCs.csv", RCM_PCs, delimiter=",")
numpy.savetxt("CanRCM4_eig.csv", RCM_eigvecs, delimiter=",")
numpy.savetxt("CanRCM4_fracVar.csv", RCM_fracVar, delimiter=",")

In [ ]:
#PCA on the HRDPS data 
data = P_hrdps.T

n_modes = np.min(np.shape(data))
pca = PCA(n_components = n_modes)
hrdps_PCs = pca.fit_transform(data)
hrdps_eigvecs = pca.components_
hrdps_fracVar = pca.explained_variance_ratio_

In [ ]:
#export the relevant arrays to CSV
numpy.savetxt("HRDPS_PCs.csv", hrdps_PCs, delimiter=",")
numpy.savetxt("HRDPS_eig.csv", hrdps_eigvecs, delimiter=",")
numpy.savetxt("HRDPS_fracVar.csv", hrdps_fracVar, delimiter=",")